In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [01:52, 11291.63it/s]
  2%|▏         | 26715/1271460 [00:00<00:04, 267089.12it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:04<00:00, 286425.91it/s]


In [3]:
def CNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    char_cnn = api_embedding_layer(char_input)
    
    kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
    pooled_word_q1 = []
    pooled_word_q2 = []
    
    for kernel in kernel_sizes:
            
        conv_word_q1 = Conv1D(filters=200,
                          kernel_size=kernel,
                          padding='same',
                          strides=1, 
                          kernel_initializer='he_uniform',
                          activation='sigmoid')(char_cnn)

        pool_word_q1 = MaxPooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        pool_word_q2 = AveragePooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        
        pooled_word_q1.append(pool_word_q1)
        pooled_word_q2.append(pool_word_q2)
        
    merged_cnn_word_q1 = Concatenate(axis=-1)(pooled_word_q1)
    merged_cnn_word_q2 = Concatenate(axis=-1)(pooled_word_q2)
    
    flatten_cnn_word_q1 = Flatten()(merged_cnn_word_q1)
    flatten_cnn_word_q2 = Flatten()(merged_cnn_word_q2)
    
    max_sub_char_abs = Lambda(lambda x:K.abs(x[0] - x[1]))([flatten_cnn_word_q1, flatten_cnn_word_q2])
    max_multi_char = Lambda(lambda x: x[0] * x[1])([flatten_cnn_word_q1, flatten_cnn_word_q2])
    
    merged = concatenate([flatten_cnn_word_q1, flatten_cnn_word_q2, \
                          max_sub_char_abs, max_multi_char])
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [4]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [5]:
del train, test, all_word
gc.collect()

48

In [7]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 128
    bst_model_path = '10folds_' + 'base_cnn_wrod_v2_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = CNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = CNN1()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 279s 3ms/step - loss: 1.1882 - acc: 0.6585 - val_loss: 0.8322 - val_acc: 0.7551
Epoch 2/20
92040/92040 [==============================] - 278s 3ms/step - loss: 0.7985 - acc: 0.7623 - val_loss: 0.7961 - val_acc: 0.7583
Epoch 3/20
92040/92040 [==============================] - 277s 3ms/step - loss: 0.6522 - acc: 0.8019 - val_loss: 0.7682 - val_acc: 0.7740
Epoch 4/20
92040/92040 [==============================] - 278s 3ms/step - loss: 0.5160 - acc: 0.8394 - val_loss: 0.8018 - val_acc: 0.7701
Epoch 5/20
92040/92040 [==============================] - 278s 3ms/step - loss: 0.3848 - acc: 0.8787 - val_loss: 0.8325 - val_acc: 0.7774
Epoch 6/20
92040/92040 [==============================] - 278s 3ms/step - loss: 0.2714 - acc: 0.9149 - val_loss: 0.9308 - val_acc: 0.7724
Epoch 7/20
102277/102277 [==============================] - 87s 851us/step
*******1*******
Train on 92043 

RuntimeError: Problems closing file (File write failed: time = tue sep  4 15:16:35 2018
, filename = '10folds_base_cnn_wrod_v2_300dim8.hdf5', file descriptor = 44, errno = 28, error message = 'no space left on device', buf = 0x55951aaaea80, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 4096)

In [8]:
%%time
#logloss_list = []
for i in [8, 9]:
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 128
    bst_model_path = '10folds_' + 'base_cnn_wrod_v2_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = CNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = CNN1()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******8*******
Train on 92056 samples, validate on 10221 samples
Epoch 1/20
92056/92056 [==============================] - 274s 3ms/step - loss: 1.1922 - acc: 0.6596 - val_loss: 0.8081 - val_acc: 0.7578
Epoch 2/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.7971 - acc: 0.7622 - val_loss: 0.7521 - val_acc: 0.7696
Epoch 3/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.6507 - acc: 0.8026 - val_loss: 0.7548 - val_acc: 0.7755
Epoch 4/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.5160 - acc: 0.8383 - val_loss: 0.7691 - val_acc: 0.7778
Epoch 5/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.3769 - acc: 0.8808 - val_loss: 0.8123 - val_acc: 0.7817
Epoch 6/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.2705 - acc: 0.9144 - val_loss: 0.8871 - val_acc: 0.7789
Epoch 7/20
92056/92056 [==============================] - 275s 3ms/step - loss: 0.2071 - acc: 0.9359 - val

In [9]:
meta_train.columns = ['meta_base_cnn_word_v2_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [10]:
p.shape, meta_train.shape

((102277, 19), (102277, 19))

In [11]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_cnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_base_cnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

In [13]:
test = pd.read_csv('test_set.csv')

pred = pd.DataFrame(test['id'])

pred['class'] = np.argmax(p.values, axis=1) + 1

pred.to_csv('/home/libo/daguan/stack_new/9.4.1.csv', index = None ,encoding = 'utf-8')

In [15]:
pred

,id,class
0,0,5
1,1,4
2,2,13
3,3,4
4,4,5
5,5,5
6,6,15
7,7,19
8,8,3
9,9,12
